# Analisi - Grafi

In [9]:
import pickle

import geopandas as gpd
import networkx as nx
from networkx.algorithms.approximation import steiner_tree
import osmnx as ox

from my_paths import *
import Graph_functions

## Sport e tempo libero

Utilizzeremo i seguenti dati:
1. **Parchi**
2. **Impianti sportivi**
3. **Aree gioco**
Saranno i nostri "poi", cioè "Point of Interest" che verranno aggiunti al Grafo della rete Ciclabile/Stradale per la ricerca di percorsi

Carichiamo il Grafo di base della rete stradale/ciclabile

In [10]:
# Carichiamo Graph Ciclabili
with open(PATH_RETE_CICLABILE_COMPLETA_PICKLE_CLEAN, "rb") as f:
    G_strade_ciclabili = pickle.load(f)

In [11]:
G_compose = G_strade_ciclabili.copy()

### Aggiunta di **Parchi** come **"poi"**

1. Carichiamo il file dei parchi da "clean"
2. Utilizziamo **"connect_poi_nodes_to_graph()"** per connettere il geodataframe come "poi" al grafo delle strade/ciclabili.

In [12]:
gdf_parchi = gpd.read_file(PATH_PARCHI_CLEAN)
G_compose = Graph_functions.connect_poi_nodes_to_graph(G_compose, gdf_parchi,
                                                                     tipo = "parchi",
                                                                     poi = True,
                                                                     artificial=True)

### Aggiunta di **Impianti sportivi** come **"poi"**

In [13]:
gdf_impianti_sportivi = gpd.read_file(PATH_IMPIANTI_SPORTIVI_CLEAN)
G_compose = Graph_functions.connect_poi_nodes_to_graph(G_compose, gdf_impianti_sportivi,
                                                                     tipo = "impianti sportivi",
                                                                     poi = True,
                                                                     artificial=True)

### Aggiunta di **Aree gioco** come **"poi"**

In [14]:
gdf_aree_gioco = gpd.read_file(PATH_AREE_GIOCO_CLEAN)
G_compose = Graph_functions.connect_poi_nodes_to_graph(G_compose, gdf_aree_gioco,
                                                                     tipo = "aree gioco",
                                                                     poi = True,
                                                                     artificial=True)

## Utilizziamo algoritmo di **Steiner Tree**

Una volta aggiunti i "poi" al grafo delle strade/ciclabili, eseguiamo l'algoritmo che troverà i percorsi minimi per connettere tutti i "poi" tra di loro con il minor "peso" possibile.  
I pesi sono stati già accuratamente inseriti sugli archi delle strade, mentre quelli delle ciclabili e degli archi all'interno dei poligoni dei "poi" sono = 0, in modo da preferire sempre quei percorsi quando possibile (sfruttare ciclabili pre-esistenti il più possibile).

In [15]:
poi_nodes = [n for n, d in G_compose.nodes(data=True) if d.get("poi") == True]
G_sport_tempo_libero = nx.MultiDiGraph(steiner_tree(G_compose, terminal_nodes=poi_nodes, weight='weight'))
# Sembra ridontante castare a MultiDiGraph il risultato di steiner_tree() che è comunque un MultiDiGraph
# ma per vari motivi non si riesce poi a salvare in pickle e quindi meglio "rifare" una copia profonda in uno nuovo

Risultato lo salviamo in Clean di Analisi (per ora anche il pickle in staging) in geojson pronto da visualizzare

In [16]:
with open(PATH_SPORT_TEMPO_LIBERO_ANALISI_PICKLE_STAGING, "wb") as f:
    pickle.dump(G_sport_tempo_libero, f)

gdf_steiner = ox.graph_to_gdfs(G_sport_tempo_libero, edges=True, nodes=False)
gdf_steiner.to_file(PATH_SPORT_TEMPO_LIBERO_ANALISI_CLEAN, driver="GeoJSON")